In [6]:
import sys
import os

# Add project root to Python path
sys.path.append(os.path.abspath(".."))

In [7]:
from agents.central_hub import CentralHub
from agents.data_prep_agent import DataPrepAgent
from agents.modelling_agent import ModellingAgent
from agents.review_agent import ReviewAgent
from agents.explanation_agent import ExplanationAgent

In [9]:
# Initialize hub and agents
hub = CentralHub()

data_prep = DataPrepAgent("Data Prep Agent", hub)
modelling = ModellingAgent("Modelling Agent", hub)
reviewing = ReviewAgent("Review Agent", hub)
explanation = ExplanationAgent("Explanation Agent", hub)

In [10]:
# Test simple communication
response = hub.send("Hub", data_prep, {"task": "clean dataset"})
print(response)

{'agent': 'Data Prep Agent', 'status': 'received', 'input': {'task': 'clean dataset'}}


In [11]:
hub.show_logs()

[2025-10-03T19:47:01.430683] Hub -> <agents.data_prep_agent.DataPrepAgent object at 0x0000020168D841C0>: {'task': 'clean dataset'}
